# Imports

In [149]:
#import the necessary libraries
from math import sqrt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary
#The following imports are used for Bayesian Optimization
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import GridSearchCV
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fetch Data (Load CIFAR 10 and apply data augementation)

In [141]:
def fetch_data():
    # Load the CIFAR-10 dataset and apply data augmentation
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
    ])

    # Load the CIFAR-10 dataset
    train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

    # Split train set into train and validation sets
    train_size = int(0.8 * len(train_set))
    val_size = len(train_set) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(train_set, [train_size, val_size])

    # Create data loaders for train, validation, and test sets
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)
    test_data_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

    return train_data_loader, val_data_loader, test_data_loader


# DEFINE RESNET Architecture

In [142]:

# Define the ResNet architecture
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear1 = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, kernel_size=3):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, kernel_size))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        return out

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, kernel_size=3):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=kernel_size, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=kernel_size,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# DEFINE UTILITY FUNCTIONS

In [143]:
def train(model, optimizer, criterion, trainloader):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item()

    train_accuracy = 100. * train_correct / train_total
    train_loss /= len(trainloader)
    print('Training - Loss: {:.4f}, Accuracy: {:.4f}%'.format(train_loss, train_accuracy))

    return train_loss, train_accuracy


def evaluate(model, criterion, testloader):
    model.eval()
    total = 0
    correct = 0
    test_loss = 0.0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss += loss.item() * labels.size(0)

    test_accuracy = 100 * correct / total
    test_loss /= len(testloader.dataset)
    
    print('Validation - Loss: {:.4f}, Accuracy: {:.4f}%'.format(test_loss , test_accuracy))
   
    
    scheduler.step(test_accuracy)

    return test_loss, test_accuracy


def train_and_validate(model, optimizer, criterion, trainloader, valloader, num_epochs):
    model.to(device)

    train_loss = 0.0
    train_accuracy=0.0
    
    val_total = 0
    val_accuracy=0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        train_loss,train_accuracy = train(model,optimizer,criterion,trainloader)
        val_loss,val_accuracy = evaluate(model,criterion,valloader)
        
       

    return train_loss, train_accuracy, val_loss, val_accuracy

def test(model,criterion,testloader):
    
    model.eval()
    total = 0
    correct = 0
    test_loss = 0.0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss += loss.item() * labels.size(0)

    test_accuracy = 100 * correct / total
    test_loss /= len(testloader.dataset)
    
    print('Testing - Loss: {:.4f}, Accuracy: {:.4f}%'.format(test_loss , test_accuracy))
    
    

    return test_loss, test_accuracy


In [144]:
# Load the CIFAR-10 dataset 
trainloader,valloader,testloader = fetch_data()
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


# 1st Experiment:
Hyperparameter Sweep

Number of Epochs = 40

Scheduler = False

Optimizer = Adam

In [89]:

# Define the search space for the hyperparameters
search_space = [Real(1e-6, 1e+0, prior='log-uniform', name='lr'),
                Real(0.1, 0.999, name='beta1'),
                Real(0.1, 0.999, name='beta2'),
                Real(1e-6, 1e-3, prior='log-uniform', name='weight_decay')]

n_epoch = 40

# Define the objective function for the Bayesian optimization
@use_named_args(search_space)
def objective_ADAM(lr, beta1, beta2, weight_decay):

    # Define the model
    model = ResNet(BasicBlock, [2, 1, 1, 1])

    print(f'\n Testing hyperparameters: lr={lr}, beta1={beta1}, beta2={beta2}, weight_decay={weight_decay}')


    # Define the optimizer with the given hyperparameters
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)

    # Train the model and return the validation accuracy
    train_losses, train_accuracies, val_losses, val_accuracies = train_and_validate(model, optimizer, criterion, trainloader, valloader, n_epoch)


    return -val_accuracies  # minimize negative accuracy

# Perform the Bayesian optimization
result = gp_minimize(objective_ADAM, search_space, n_calls=10, random_state=0)

# Print the best hyperparameters and the corresponding validation accuracy
print('Best hyperparameters:', result.x)
print('Best validation accuracy:', -result.fun)



 Testing hyperparameters: lr=0.0036063486397284665, beta1=0.8589949079743348, beta2=0.8712931102428585, weight_decay=0.0003481410351917474
Epoch 1/40
----------
Train - Batch: 0/625, Loss: 6.9230, Accuracy: 0.0000%
Train - Batch: 100/625, Loss: 2.0532, Accuracy: 25.6962%
Train - Batch: 200/625, Loss: 0.9231, Accuracy: 28.8713%
Train - Batch: 300/625, Loss: 0.5847, Accuracy: 30.8036%
Train - Batch: 400/625, Loss: 0.4209, Accuracy: 32.7229%
Train - Batch: 500/625, Loss: 0.3237, Accuracy: 34.3282%
Train - Batch: 600/625, Loss: 0.2628, Accuracy: 35.7061%
Validation - Batch: 156/157, Loss: 1.5741, Accuracy: 41.7900%
Epoch 2/40
----------
Train - Batch: 0/625, Loss: 1.1918, Accuracy: 53.1250%
Train - Batch: 100/625, Loss: 1.5094, Accuracy: 44.5080%
Train - Batch: 200/625, Loss: 0.7258, Accuracy: 46.1365%
Train - Batch: 300/625, Loss: 0.4650, Accuracy: 47.2488%
Train - Batch: 400/625, Loss: 0.3517, Accuracy: 47.5218%
Train - Batch: 500/625, Loss: 0.2749, Accuracy: 48.1506%
Train - Batch: 600

In [90]:
# Extract the best hyperparameters found by Bayesian optimization
best_lr, best_beta1, best_beta2, best_weight_decay = result.x

# Define the model and dataset
model = ResNet(BasicBlock, [2, 1, 1, 1])

# Define the Adam optimizer with the best hyperparameters
optimizer = optim.Adam(model.parameters(), lr=best_lr, betas=(best_beta1, best_beta2), weight_decay=best_weight_decay)

# Train the model with the best hyperparameters and validate
train_and_validate(model,optimizer,criterion,trainloader,valloader,50)


Epoch 1/50
----------
Train - Batch: 0/625, Loss: 7.0554, Accuracy: 0.0000%
Train - Batch: 100/625, Loss: 2.1020, Accuracy: 31.1572%
Train - Batch: 200/625, Loss: 0.8182, Accuracy: 35.5955%
Train - Batch: 300/625, Loss: 0.5097, Accuracy: 38.4292%
Train - Batch: 400/625, Loss: 0.3684, Accuracy: 40.3328%
Train - Batch: 500/625, Loss: 0.2849, Accuracy: 42.0097%
Train - Batch: 600/625, Loss: 0.2268, Accuracy: 43.3886%
Validation - Batch: 156/157, Loss: 1.3395, Accuracy: 52.6600%
Epoch 2/50
----------
Train - Batch: 0/625, Loss: 1.3195, Accuracy: 50.0000%
Train - Batch: 100/625, Loss: 1.2723, Accuracy: 53.7438%
Train - Batch: 200/625, Loss: 0.6188, Accuracy: 54.7030%
Train - Batch: 300/625, Loss: 0.4028, Accuracy: 55.3623%
Train - Batch: 400/625, Loss: 0.2992, Accuracy: 55.9811%
Train - Batch: 500/625, Loss: 0.2348, Accuracy: 56.4933%
Train - Batch: 600/625, Loss: 0.1982, Accuracy: 56.7752%
Validation - Batch: 156/157, Loss: 1.1500, Accuracy: 59.3600%
Epoch 3/50
----------
Train - Batch: 0/

(575.4851904989425, 86.77984608985025, 0.5857106620788575, 81.24)

In [110]:
#Test the model on TestDataSet
test(model,criterion,testloader)


Test set: Average loss: 0.0468, Accuracy: 4492/10000 (45%)



(0.04682503364086151, 44.92)

# Train the model with the best parameters obtained but while using a learning rate scheduler

In [104]:
# Define the best hyperparameters found by Bayesian optimization
best_lr, best_beta1, best_beta2, best_weight_decay = result.x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model and dataset
model = ResNet(BasicBlock, [2, 1, 1, 1])
# Define the Adam optimizer with the best hyperparameters
optimizer = optim.Adam(model.parameters(), lr=best_lr, betas=(best_beta1, best_beta2), weight_decay=best_weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max')


# Train the model with the best hyperparameters and validate
train_and_validate(model,optimizer,criterion,trainloader,valloader,50)

Epoch 1/50
----------
Train - Batch: 0/625, Loss: 7.2310, Accuracy: 0.0000%
Train - Batch: 100/625, Loss: 2.0870, Accuracy: 31.6986%
Train - Batch: 200/625, Loss: 0.8445, Accuracy: 34.9736%
Train - Batch: 300/625, Loss: 0.5224, Accuracy: 37.5260%
Train - Batch: 400/625, Loss: 0.3665, Accuracy: 39.6470%
Train - Batch: 500/625, Loss: 0.2860, Accuracy: 41.4795%
Train - Batch: 600/625, Loss: 0.2223, Accuracy: 43.1780%
Validation - Batch: 156/157, Loss: 1.3825, Accuracy: 50.2200%
Epoch 2/50
----------
Train - Batch: 0/625, Loss: 1.4223, Accuracy: 54.6875%
Train - Batch: 100/625, Loss: 1.2747, Accuracy: 54.0996%
Train - Batch: 200/625, Loss: 0.6208, Accuracy: 54.9596%
Train - Batch: 300/625, Loss: 0.4029, Accuracy: 55.5025%
Train - Batch: 400/625, Loss: 0.3054, Accuracy: 55.7357%
Train - Batch: 500/625, Loss: 0.2366, Accuracy: 56.2406%
Train - Batch: 600/625, Loss: 0.1893, Accuracy: 56.8402%
Validation - Batch: 156/157, Loss: 1.1507, Accuracy: 59.6100%
Epoch 3/50
----------
Train - Batch: 0/

(634.9966974731038, 87.06322795341099, 0.552321842956543, 82.35)

In [117]:
test(model,criterion,testloader)


Test set: Average loss: 0.0468, Accuracy: 4492/10000 (45%)

(0.04681235432624817, 44.92)


# Second Run with changes in data splitting and train function

In [132]:
#Second RUN with changes in data splitting and train function

# Define the search space for the hyperparameters
search_space = [Real(1e-6, 1e+0, prior='log-uniform', name='lr'),
                Real(0.1, 0.999, name='beta1'),
                Real(0.1, 0.999, name='beta2'),
                Real(1e-6, 1e-3, prior='log-uniform', name='weight_decay')]

n_epoch = 40

# Define the objective function for the Bayesian optimization
@use_named_args(search_space)
def objective_ADAM(lr, beta1, beta2, weight_decay):

    # Define the model
    model = ResNet(BasicBlock, [2, 1, 1, 1])

    print(f'\n Testing hyperparameters: lr={lr}, beta1={beta1}, beta2={beta2}, weight_decay={weight_decay}')


    # Define the optimizer with the given hyperparameters
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)

    # Train the model and return the validation accuracy
    train_losses, train_accuracies, val_losses, val_accuracies = train_and_validate(model, optimizer, criterion, trainloader, valloader, n_epoch)


    return -val_accuracies  # minimize negative accuracy

# Perform the Bayesian optimization
result = gp_minimize(objective_ADAM, search_space, n_calls=10, random_state=0)

# Print the best hyperparameters and the corresponding validation accuracy
print('Best hyperparameters:', result.x)
print('Best validation accuracy:', -result.fun)



 Testing hyperparameters: lr=0.0036063486397284665, beta1=0.8589949079743348, beta2=0.8712931102428585, weight_decay=0.0003481410351917474
Epoch 1/40
----------
Training - Loss: 1.8402, Accuracy: 32.9600%
Validation - Loss: 1.6077, Accuracy: 42.5600%
Epoch 2/40
----------
Training - Loss: 1.4791, Accuracy: 46.4825%
Validation - Loss: 1.3257, Accuracy: 51.7500%
Epoch 3/40
----------
Training - Loss: 1.3262, Accuracy: 52.4650%
Validation - Loss: 1.2295, Accuracy: 55.9000%
Epoch 4/40
----------
Training - Loss: 1.2456, Accuracy: 55.8425%
Validation - Loss: 1.2820, Accuracy: 55.2500%
Epoch 5/40
----------
Training - Loss: 1.1798, Accuracy: 58.2375%
Validation - Loss: 1.1019, Accuracy: 61.2500%
Epoch 6/40
----------
Training - Loss: 1.1447, Accuracy: 59.6375%
Validation - Loss: 1.1108, Accuracy: 60.9200%
Epoch 7/40
----------
Training - Loss: 1.1204, Accuracy: 60.7125%
Validation - Loss: 1.1284, Accuracy: 60.6900%
Epoch 8/40
----------
Training - Loss: 1.0955, Accuracy: 61.7000%
Validation

In [156]:
# Define the best hyperparameters found by Bayesian optimization
best_lr, best_beta1, best_beta2, best_weight_decay = result.x

# Define the model and dataset
model = ResNet(BasicBlock, [2, 1, 1, 1])
summary(model.to(device), input_size=(3, 32, 32))
# Define the Adam optimizer with the best hyperparameters
optimizer = optim.Adam(model.parameters(), lr=best_lr, betas=(best_beta1, best_beta2), weight_decay=best_weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',patience=3)


# Train the model with the best hyperparameters and validate
train_and_validate(model,optimizer,criterion,trainloader,valloader,100)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

(0.059744104460068045, 98.16, 0.26453124999701977, 91.69)

In [157]:
test(model,criterion,testloader)

Test set: Average loss: 0.0092, Accuracy: 9165/10000 (92%)



(0.009164879802265205, 91.65)

# Perform Bayesian Optimization with SGD

In [ ]:
# Define the search space for the hyperparameters
search_space = [Real(1e-6, 0.4, prior='log-uniform', name='lr'),
                Real(0.1, 0.999, name='momentum'),
                Real(1e-6, 1e-3, prior='log-uniform', name='weight_decay')]

n_epoch = 40

# Define the objective function for the Bayesian optimization
@use_named_args(search_space)
def objective_SGD(lr, momentum, weight_decay):

    # Define the model
    model = ResNet(BasicBlock, [2, 1, 1, 1])

    print(f'\n Testing hyperparameters: lr={lr}, momentum={momentum}, weight_decay={weight_decay}')
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',patience=3)
    # Define the optimizer with the given hyperparameters
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    # Train the model and return the validation accuracy
    train_losses, train_accuracies, val_losses, val_accuracies = train_and_validate(model, optimizer, criterion, trainloader, valloader, n_epoch)


    return -val_accuracies  # minimize negative accuracy

# Perform the Bayesian optimization
result = gp_minimize(objective_SGD, search_space, n_calls=10, random_state=0)

# Print the best hyperparameters and the corresponding validation accuracy
print('Best hyperparameters:', result.x)
print('Best validation accuracy:', -result.fun)


Testing hyperparameters: lr=0.0020948412486806525, momentum=0.8589949079743348, weight_decay=0.0003748321662847933
Epoch 1/5
----------
Training - Loss: 1.5059, Accuracy: 44.7850%
Validation - Loss: 1.7939, Accuracy: 45.1800%
Testing - Loss: 1.7939, Accuracy: 45.1800%
Epoch 2/5
----------
Training - Loss: 1.0865, Accuracy: 61.0350%
Validation - Loss: 0.9479, Accuracy: 66.5300%
Testing - Loss: 0.9479, Accuracy: 66.5300%
Epoch 3/5
----------
Training - Loss: 0.9037, Accuracy: 68.1600%
Validation - Loss: 0.9000, Accuracy: 68.5500%
Testing - Loss: 0.9000, Accuracy: 68.5500%
Epoch 4/5
----------
Training - Loss: 0.7832, Accuracy: 72.6400%
Validation - Loss: 0.7745, Accuracy: 73.3900%
Testing - Loss: 0.7745, Accuracy: 73.3900%
Epoch 5/5
----------
Training - Loss: 0.6956, Accuracy: 75.9125%
Validation - Loss: 0.7044, Accuracy: 75.6900%
Testing - Loss: 0.7044, Accuracy: 75.6900%

 Testing hyperparameters: lr=0.055764147648798994, momentum=0.6605837634105892, weight_decay=1.4228041454890752e-05
Epoch 1/5
----------
Training - Loss: 1.5219, Accuracy: 43.9025%
Validation - Loss: 1.2289, Accuracy: 55.7200%
Testing - Loss: 1.2289, Accuracy: 55.7200%
Epoch 2/5
----------
Training - Loss: 1.0468, Accuracy: 62.4600%
Validation - Loss: 1.2107, Accuracy: 59.4700%
Testing - Loss: 1.2107, Accuracy: 59.4700%
Epoch 3/5
----------
Training - Loss: 0.8348, Accuracy: 70.5675%
Validation - Loss: 0.8424, Accuracy: 70.6000%
Testing - Loss: 0.8424, Accuracy: 70.6000%
Epoch 4/5
----------
Training - Loss: 0.7063, Accuracy: 75.3275%
Validation - Loss: 0.7217, Accuracy: 75.2100%
Testing - Loss: 0.7217, Accuracy: 75.2100%
Epoch 5/5
----------
Training - Loss: 0.6139, Accuracy: 78.6600%
Validation - Loss: 0.6316, Accuracy: 78.4000%
Testing - Loss: 0.6316, Accuracy: 78.4000%

 Testing hyperparameters: lr=4.6430858668320706e-05, momentum=0.15098496660838145, weight_decay=6.576107192827546e-06
Epoch 1/5
----------
Training - Loss: 2.2924, Accuracy: 13.5325%
Validation - Loss: 2.2203, Accuracy: 19.5100%
Testing - Loss: 2.2203, Accuracy: 19.5100%
Epoch 2/5
----------
Training - Loss: 2.1813, Accuracy: 21.0200%
Validation - Loss: 2.1280, Accuracy: 23.9200%
Testing - Loss: 2.1280, Accuracy: 23.9200%
Epoch 3/5
----------
Training - Loss: 2.1088, Accuracy: 23.5225%
Validation - Loss: 2.0625, Accuracy: 26.1600%
Testing - Loss: 2.0625, Accuracy: 26.1600%
Epoch 4/5
----------
Training - Loss: 2.0555, Accuracy: 25.5075%
Validation - Loss: 2.0169, Accuracy: 27.3800%
Testing - Loss: 2.0169, Accuracy: 27.3800%
Epoch 5/5
----------
Training - Loss: 2.0134, Accuracy: 27.2425%
Validation - Loss: 1.9768, Accuracy: 28.7900%
Testing - Loss: 1.9768, Accuracy: 28.7900%

 Testing hyperparameters: lr=0.00047414223550336436, momentum=0.8301396871691685, weight_decay=2.753794429736146e-05
Epoch 1/5
----------
Training - Loss: 1.7265, Accuracy: 36.5100%
Validation - Loss: 1.5015, Accuracy: 44.8200%
Testing - Loss: 1.5015, Accuracy: 44.8200%
Epoch 2/5
----------
Training - Loss: 1.3823, Accuracy: 49.5950%
Validation - Loss: 1.3642, Accuracy: 51.7800%
Testing - Loss: 1.3642, Accuracy: 51.7800%
Epoch 3/5
----------
Training - Loss: 1.2061, Accuracy: 56.7350%
Validation - Loss: 1.1694, Accuracy: 57.9000%
Testing - Loss: 1.1694, Accuracy: 57.9000%
Epoch 4/5
----------
Training - Loss: 1.0848, Accuracy: 61.3850%
Validation - Loss: 1.0374, Accuracy: 63.6100%
Testing - Loss: 1.0374, Accuracy: 63.6100%
Epoch 5/5
----------
Training - Loss: 0.9892, Accuracy: 64.6250%
Validation - Loss: 0.9750, Accuracy: 65.8300%
Testing - Loss: 0.9750, Accuracy: 65.8300%

 Testing hyperparameters: lr=0.00015863681851289703, momentum=0.8516348084201026, weight_decay=1.028462547983764e-05
Epoch 1/5
----------
Training - Loss: 1.8938, Accuracy: 31.0075%
Validation - Loss: 1.6730, Accuracy: 38.5300%
Testing - Loss: 1.6730, Accuracy: 38.5300%
Epoch 2/5
----------
Training - Loss: 1.6003, Accuracy: 41.1850%
Validation - Loss: 1.5221, Accuracy: 44.0700%
Testing - Loss: 1.5221, Accuracy: 44.0700%
Epoch 3/5
----------
Training - Loss: 1.4629, Accuracy: 46.9500%
Validation - Loss: 1.4045, Accuracy: 49.0500%
Testing - Loss: 1.4045, Accuracy: 49.0500%
Epoch 4/5
----------
Training - Loss: 1.3591, Accuracy: 50.9500%
Validation - Loss: 1.3029, Accuracy: 52.8300%
Testing - Loss: 1.3029, Accuracy: 52.8300%
Epoch 5/5
----------
Training - Loss: 1.2761, Accuracy: 53.9425%
Validation - Loss: 1.2221, Accuracy: 56.8800%
Testing - Loss: 1.2221, Accuracy: 56.8800%

 Testing hyperparameters: lr=0.004276590943544507, momentum=0.43104914431665264, weight_decay=0.0007438159327920417
Epoch 1/5
----------
Training - Loss: 1.5570, Accuracy: 42.7775%
Validation - Loss: 1.3453, Accuracy: 51.6000%
Testing - Loss: 1.3453, Accuracy: 51.6000%
Epoch 2/5
----------
Training - Loss: 1.1622, Accuracy: 57.9425%
Validation - Loss: 1.0647, Accuracy: 62.0600%
Testing - Loss: 1.0647, Accuracy: 62.0600%
Epoch 3/5
----------
Training - Loss: 0.9741, Accuracy: 65.1075%
Validation - Loss: 1.0051, Accuracy: 64.3100%
Testing - Loss: 1.0051, Accuracy: 64.3100%
Epoch 4/5
----------
Training - Loss: 0.8618, Accuracy: 69.4775%
Validation - Loss: 0.8595, Accuracy: 69.3000%
Testing - Loss: 0.8595, Accuracy: 69.3000%
Epoch 5/5
----------
Training - Loss: 0.7741, Accuracy: 72.7300%
Validation - Loss: 0.7402, Accuracy: 74.5300%
Testing - Loss: 0.7402, Accuracy: 74.5300%

 Testing hyperparameters: lr=6.1129872743331135e-06, momentum=0.8822084452642345, weight_decay=2.6352640400187277e-05
Epoch 1/5
----------
Training - Loss: 2.2915, Accuracy: 12.8775%
Validation - Loss: 2.2445, Accuracy: 17.1500%
Testing - Loss: 2.2445, Accuracy: 17.1500%
Epoch 2/5
----------
Training - Loss: 2.2149, Accuracy: 19.2000%
Validation - Loss: 2.1726, Accuracy: 22.0500%
Testing - Loss: 2.1726, Accuracy: 22.0500%
Epoch 3/5
----------
Training - Loss: 2.1484, Accuracy: 22.8700%
Validation - Loss: 2.1016, Accuracy: 24.8700%
Testing - Loss: 2.1016, Accuracy: 24.8700%
Epoch 4/5
----------
Training - Loss: 2.0907, Accuracy: 24.6475%
Validation - Loss: 2.0535, Accuracy: 26.4300%
Testing - Loss: 2.0535, Accuracy: 26.4300%
Epoch 5/5
----------
Training - Loss: 2.0440, Accuracy: 26.1950%
Validation - Loss: 2.0078, Accuracy: 28.1300%
Testing - Loss: 2.0078, Accuracy: 28.1300%

 Testing hyperparameters: lr=0.03067256392766774, momentum=0.5679092541165333, weight_decay=0.00010880242882938465
Epoch 1/5
----------
Training - Loss: 1.4762, Accuracy: 45.6025%
Validation - Loss: 1.2359, Accuracy: 55.9600%
Testing - Loss: 1.2359, Accuracy: 55.9600%
Epoch 2/5
----------
Training - Loss: 1.0161, Accuracy: 63.9125%
Validation - Loss: 0.9299, Accuracy: 66.9100%
Testing - Loss: 0.9299, Accuracy: 66.9100%
Epoch 3/5
----------
Training - Loss: 0.8126, Accuracy: 71.3250%
Validation - Loss: 1.0909, Accuracy: 65.0200%
Testing - Loss: 1.0909, Accuracy: 65.0200%
Epoch 4/5
----------
Training - Loss: 0.6967, Accuracy: 75.5100%
Validation - Loss: 0.6782, Accuracy: 75.9900%
Testing - Loss: 0.6782, Accuracy: 75.9900%
Epoch 5/5
----------
Training - Loss: 0.6079, Accuracy: 78.9350%
Validation - Loss: 0.7309, Accuracy: 75.0200%
Testing - Loss: 0.7309, Accuracy: 75.0200%

 Testing hyperparameters: lr=0.010889987054427588, momentum=0.6232357930755213, weight_decay=4.093734644841746e-05
Epoch 1/5
----------
Training - Loss: 1.4503, Accuracy: 46.5850%
Validation - Loss: 1.1988, Accuracy: 57.1700%
Testing - Loss: 1.1988, Accuracy: 57.1700%
Epoch 2/5
----------
Training - Loss: 1.0204, Accuracy: 63.5825%
Validation - Loss: 0.9017, Accuracy: 68.2900%
Testing - Loss: 0.9017, Accuracy: 68.2900%
Epoch 3/5
----------
Training - Loss: 0.8389, Accuracy: 70.3675%
Validation - Loss: 0.9163, Accuracy: 67.5500%
Testing - Loss: 0.9163, Accuracy: 67.5500%
Epoch 4/5
----------
Training - Loss: 0.7238, Accuracy: 74.8975%
Validation - Loss: 0.6770, Accuracy: 76.6700%
Testing - Loss: 0.6770, Accuracy: 76.6700%
Epoch 5/5
----------
Training - Loss: 0.6384, Accuracy: 77.6400%
Validation - Loss: 0.7176, Accuracy: 76.1100%
Testing - Loss: 0.7176, Accuracy: 76.1100%

 Testing hyperparameters: lr=0.017775025719896944, momentum=0.19521093886182517, weight_decay=2.6351252231361942e-05
 
Epoch 1/5
----------
Training - Loss: 1.4502, Accuracy: 46.5100%
Validation - Loss: 1.2017, Accuracy: 56.5600%
Testing - Loss: 1.2017, Accuracy: 56.5600%
Epoch 2/5
----------
Training - Loss: 1.0265, Accuracy: 63.2025%
Validation - Loss: 1.0600, Accuracy: 62.4500%
Testing - Loss: 1.0600, Accuracy: 62.4500%
Epoch 3/5
----------
Training - Loss: 0.8440, Accuracy: 70.2225%
Validation - Loss: 0.8211, Accuracy: 71.0200%
Testing - Loss: 0.8211, Accuracy: 71.0200%
Epoch 4/5
----------
Training - Loss: 0.7230, Accuracy: 74.7250%
Validation - Loss: 0.7284, Accuracy: 74.4200%
Testing - Loss: 0.7284, Accuracy: 74.4200%
Epoch 5/5
----------
Training - Loss: 0.6435, Accuracy: 77.4750%
Validation - Loss: 0.7015, Accuracy: 75.7500%
Testing - Loss: 0.7015, Accuracy: 75.7500%
Best hyperparameters: [0.055764147648798994, 0.6605837634105892, 1.4228041454890752e-05]
Best validation accuracy: 78.4


In [ ]:
# Define the best hyperparameters found by Bayesian optimization
best_lr, best_momentum, best_weight_decay = [0.0015078369731868298, 0.8516348084201026, 0.001028462547983764]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model and dataset
model = ResNet(BasicBlock, [2, 1, 1, 1])
# Define the SGD optimizer with the best hyperparameters
optimizer = optim.SGD(model.parameters(), lr=best_lr, momentum=best_momentum, weight_decay=best_weight_decay)
#optimizer = optim.Adam(model_5.parameters(), lr=0.1, betas=(0.9,0.9), weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max')
criterion = nn.CrossEntropyLoss()

# Train the model with the best hyperparameters and validate
train_and_validate(model,optimizer,criterion,trainloader,valloader,100)

Epoch 1/100
----------
Training - Loss: 1.5438, Accuracy: 43.0325%
Validation - Loss: 1.3433, Accuracy: 51.8800%
Testing - Loss: 1.3433, Accuracy: 51.8800%
Epoch 2/100
----------
Training - Loss: 1.1482, Accuracy: 58.9775%
Validation - Loss: 1.0060, Accuracy: 64.2200%
Testing - Loss: 1.0060, Accuracy: 64.2200%
Epoch 3/100
----------
Training - Loss: 0.9637, Accuracy: 65.7975%
Validation - Loss: 0.9073, Accuracy: 68.1200%
Testing - Loss: 0.9073, Accuracy: 68.1200%
Epoch 4/100
----------
Training - Loss: 0.8487, Accuracy: 70.4200%
Validation - Loss: 0.7998, Accuracy: 71.8300%
Testing - Loss: 0.7998, Accuracy: 71.8300%
Epoch 5/100
----------
Training - Loss: 0.7591, Accuracy: 73.6250%
Validation - Loss: 0.7720, Accuracy: 73.4200%
Testing - Loss: 0.7720, Accuracy: 73.4200%
Epoch 6/100
----------
Training - Loss: 0.6787, Accuracy: 76.2775%
Validation - Loss: 0.6731, Accuracy: 76.3700%
Testing - Loss: 0.6731, Accuracy: 76.3700%
Epoch 7/100
----------
Training - Loss: 0.6217, Accuracy: 78.5800%
Validation - Loss: 0.6695, Accuracy: 76.6300%
Testing - Loss: 0.6695, Accuracy: 76.6300%
Epoch 8/100
----------
Training - Loss: 0.5792, Accuracy: 80.0450%
Validation - Loss: 0.5926, Accuracy: 79.5100%
Testing - Loss: 0.5926, Accuracy: 79.5100%
Epoch 9/100
----------
Training - Loss: 0.5444, Accuracy: 81.2875%
Validation - Loss: 0.5512, Accuracy: 80.7500%
Testing - Loss: 0.5512, Accuracy: 80.7500%
Epoch 10/100
----------
Training - Loss: 0.5059, Accuracy: 82.6800%
Validation - Loss: 0.5773, Accuracy: 79.7900%
Testing - Loss: 0.5773, Accuracy: 79.7900%
Epoch 11/100
----------
Training - Loss: 0.4782, Accuracy: 83.4500%
Validation - Loss: 0.5100, Accuracy: 82.3200%
Testing - Loss: 0.5100, Accuracy: 82.3200%
Epoch 12/100
----------
Training - Loss: 0.4597, Accuracy: 84.2650%
Validation - Loss: 0.4840, Accuracy: 83.5100%
Testing - Loss: 0.4840, Accuracy: 83.5100%
Epoch 13/100
----------
Training - Loss: 0.4312, Accuracy: 85.1400%
Validation - Loss: 0.4535, Accuracy: 84.5500%
Testing - Loss: 0.4535, Accuracy: 84.5500%
Epoch 14/100
----------
Training - Loss: 0.4119, Accuracy: 85.8225%
Validation - Loss: 0.4728, Accuracy: 83.9200%
Testing - Loss: 0.4728, Accuracy: 83.9200%
Epoch 15/100
----------
Training - Loss: 0.3958, Accuracy: 86.3675%
Validation - Loss: 0.4696, Accuracy: 83.6900%
Testing - Loss: 0.4696, Accuracy: 83.6900%
Epoch 16/100
----------
Training - Loss: 0.3803, Accuracy: 86.7750%
Validation - Loss: 0.5109, Accuracy: 82.7200%
Testing - Loss: 0.5109, Accuracy: 82.7200%
Epoch 17/100
----------
Training - Loss: 0.3635, Accuracy: 87.4325%
Validation - Loss: 0.4499, Accuracy: 84.4500%
Testing - Loss: 0.4499, Accuracy: 84.4500%
Epoch 18/100
----------
Training - Loss: 0.3505, Accuracy: 87.8650%
Validation - Loss: 0.4398, Accuracy: 85.0000%
Testing - Loss: 0.4398, Accuracy: 85.0000%
Epoch 19/100
----------
Training - Loss: 0.3319, Accuracy: 88.4125%
Validation - Loss: 0.4028, Accuracy: 86.2400%
Testing - Loss: 0.4028, Accuracy: 86.2400%
Epoch 20/100
----------
Training - Loss: 0.3225, Accuracy: 89.0275%
Validation - Loss: 0.3832, Accuracy: 87.1400%
Testing - Loss: 0.3832, Accuracy: 87.1400%
Epoch 21/100
----------
Training - Loss: 0.3081, Accuracy: 89.4250%
Validation - Loss: 0.3870, Accuracy: 86.7200%
Testing - Loss: 0.3870, Accuracy: 86.7200%
Epoch 22/100
----------
Training - Loss: 0.3004, Accuracy: 89.6475%
Validation - Loss: 0.4070, Accuracy: 86.1300%
Testing - Loss: 0.4070, Accuracy: 86.1300%
Epoch 23/100
----------
Training - Loss: 0.2916, Accuracy: 89.8775%
Validation - Loss: 0.3784, Accuracy: 86.7400%
Testing - Loss: 0.3784, Accuracy: 86.7400%
Epoch 24/100
----------
Training - Loss: 0.2746, Accuracy: 90.5200%
Validation - Loss: 0.3919, Accuracy: 86.7300%
Testing - Loss: 0.3919, Accuracy: 86.7300%
Epoch 25/100
----------
Training - Loss: 0.2707, Accuracy: 90.6950%
Validation - Loss: 0.3846, Accuracy: 86.9900%
Testing - Loss: 0.3846, Accuracy: 86.9900%
Epoch 26/100
----------
Training - Loss: 0.2619, Accuracy: 90.9375%
Validation - Loss: 0.3548, Accuracy: 88.1000%
Testing - Loss: 0.3548, Accuracy: 88.1000%
Epoch 27/100
----------
Training - Loss: 0.2460, Accuracy: 91.3500%
Validation - Loss: 0.3668, Accuracy: 87.6700%
Testing - Loss: 0.3668, Accuracy: 87.6700%
Epoch 28/100
----------
Training - Loss: 0.2473, Accuracy: 91.3075%
Validation - Loss: 0.3570, Accuracy: 87.6900%
Testing - Loss: 0.3570, Accuracy: 87.6900%
Epoch 29/100
----------
Training - Loss: 0.2412, Accuracy: 91.6800%
Validation - Loss: 0.3531, Accuracy: 87.3800%
Testing - Loss: 0.3531, Accuracy: 87.3800%
Epoch 30/100
----------
Training - Loss: 0.2261, Accuracy: 92.1425%
Validation - Loss: 0.3549, Accuracy: 88.1900%
Testing - Loss: 0.3549, Accuracy: 88.1900%
Epoch 31/100
----------
Training - Loss: 0.2249, Accuracy: 92.2375%
Validation - Loss: 0.3852, Accuracy: 87.0500%
Testing - Loss: 0.3852, Accuracy: 87.0500%
Epoch 32/100
----------
Training - Loss: 0.2175, Accuracy: 92.5200%
Validation - Loss: 0.3496, Accuracy: 88.3400%
Testing - Loss: 0.3496, Accuracy: 88.3400%
Epoch 33/100
----------
Training - Loss: 0.2058, Accuracy: 93.0300%
Validation - Loss: 0.3448, Accuracy: 88.6100%
Testing - Loss: 0.3448, Accuracy: 88.6100%
Epoch 34/100
----------
Training - Loss: 0.1983, Accuracy: 93.2175%
Validation - Loss: 0.3500, Accuracy: 88.3600%
Testing - Loss: 0.3500, Accuracy: 88.3600%
Epoch 35/100
----------
Training - Loss: 0.1974, Accuracy: 93.1900%
Validation - Loss: 0.3374, Accuracy: 88.6400%
Testing - Loss: 0.3374, Accuracy: 88.6400%
Epoch 36/100
----------
Training - Loss: 0.1898, Accuracy: 93.5950%
Validation - Loss: 0.3446, Accuracy: 88.8100%
Testing - Loss: 0.3446, Accuracy: 88.8100%
Epoch 37/100
----------
Training - Loss: 0.1855, Accuracy: 93.5625%
Validation - Loss: 0.3728, Accuracy: 87.9800%
Testing - Loss: 0.3728, Accuracy: 87.9800%
Epoch 38/100
----------
Training - Loss: 0.1776, Accuracy: 93.8575%
Validation - Loss: 0.3409, Accuracy: 88.3600%
Testing - Loss: 0.3409, Accuracy: 88.3600%
Epoch 39/100
----------
Training - Loss: 0.1766, Accuracy: 93.8650%
Validation - Loss: 0.3525, Accuracy: 88.6500%
Testing - Loss: 0.3525, Accuracy: 88.6500%
Epoch 40/100
----------
Training - Loss: 0.1642, Accuracy: 94.3825%
Validation - Loss: 0.3345, Accuracy: 88.9100%
Testing - Loss: 0.3345, Accuracy: 88.9100%
Epoch 41/100
----------
Training - Loss: 0.1633, Accuracy: 94.3575%
Validation - Loss: 0.3188, Accuracy: 89.6600%
Testing - Loss: 0.3188, Accuracy: 89.6600%
Epoch 42/100
----------
Training - Loss: 0.1567, Accuracy: 94.6000%
Validation - Loss: 0.3167, Accuracy: 89.6900%
Testing - Loss: 0.3167, Accuracy: 89.6900%
Epoch 43/100
----------
Training - Loss: 0.1532, Accuracy: 94.7850%
Validation - Loss: 0.3704, Accuracy: 88.2400%
Testing - Loss: 0.3704, Accuracy: 88.2400%
Epoch 44/100
----------
Training - Loss: 0.1474, Accuracy: 95.0850%
Validation - Loss: 0.3596, Accuracy: 88.5900%
Testing - Loss: 0.3596, Accuracy: 88.5900%
Epoch 45/100
----------
Training - Loss: 0.1459, Accuracy: 95.0775%
Validation - Loss: 0.3611, Accuracy: 88.3500%
Testing - Loss: 0.3611, Accuracy: 88.3500%
Epoch 46/100
----------
Training - Loss: 0.1418, Accuracy: 95.1550%
Validation - Loss: 0.3826, Accuracy: 88.1500%
Testing - Loss: 0.3826, Accuracy: 88.1500%
Epoch 47/100
----------
Training - Loss: 0.1383, Accuracy: 95.2025%
Validation - Loss: 0.3281, Accuracy: 89.4500%
Testing - Loss: 0.3281, Accuracy: 89.4500%
Epoch 48/100
----------
Training - Loss: 0.1360, Accuracy: 95.3875%
Validation - Loss: 0.3134, Accuracy: 89.8200%
Testing - Loss: 0.3134, Accuracy: 89.8200%
Epoch 49/100
----------
Training - Loss: 0.1303, Accuracy: 95.6000%
Validation - Loss: 0.3171, Accuracy: 89.6100%
Testing - Loss: 0.3171, Accuracy: 89.6100%
Epoch 50/100
----------
Training - Loss: 0.1263, Accuracy: 95.7325%
Validation - Loss: 0.3321, Accuracy: 89.5600%
Testing - Loss: 0.3321, Accuracy: 89.5600%
Epoch 51/100
----------
Training - Loss: 0.1260, Accuracy: 95.7250%
Validation - Loss: 0.3105, Accuracy: 89.8600%
Testing - Loss: 0.3105, Accuracy: 89.8600%
Epoch 52/100
----------
Training - Loss: 0.1194, Accuracy: 95.9950%
Validation - Loss: 0.3184, Accuracy: 89.6200%
Testing - Loss: 0.3184, Accuracy: 89.6200%
Epoch 53/100
----------
Training - Loss: 0.1184, Accuracy: 95.9600%
Validation - Loss: 0.3500, Accuracy: 89.1200%
Testing - Loss: 0.3500, Accuracy: 89.1200%
Epoch 54/100
----------
Training - Loss: 0.1165, Accuracy: 96.0250%
Validation - Loss: 0.3649, Accuracy: 88.5000%
Testing - Loss: 0.3649, Accuracy: 88.5000%
Epoch 55/100
----------
Training - Loss: 0.1128, Accuracy: 96.3375%
Validation - Loss: 0.3303, Accuracy: 89.4900%
Testing - Loss: 0.3303, Accuracy: 89.4900%
Epoch 56/100
----------
Training - Loss: 0.1108, Accuracy: 96.4350%
Validation - Loss: 0.3483, Accuracy: 89.0700%
Testing - Loss: 0.3483, Accuracy: 89.0700%
Epoch 57/100
----------
Training - Loss: 0.1083, Accuracy: 96.3575%
Validation - Loss: 0.3039, Accuracy: 90.3900%
Testing - Loss: 0.3039, Accuracy: 90.3900%
Epoch 58/100
----------
Training - Loss: 0.1012, Accuracy: 96.7125%
Validation - Loss: 0.3139, Accuracy: 89.7400%
Testing - Loss: 0.3139, Accuracy: 89.7400%
Epoch 59/100
----------
Training - Loss: 0.0988, Accuracy: 96.8100%
Validation - Loss: 0.3656, Accuracy: 89.0500%
Testing - Loss: 0.3656, Accuracy: 89.0500%
Epoch 60/100
----------
Training - Loss: 0.0989, Accuracy: 96.7575%
Validation - Loss: 0.3093, Accuracy: 90.1600%
Testing - Loss: 0.3093, Accuracy: 90.1600%
Epoch 61/100
----------
Training - Loss: 0.0997, Accuracy: 96.7425%
Validation - Loss: 0.3267, Accuracy: 89.9500%
Testing - Loss: 0.3267, Accuracy: 89.9500%
Epoch 62/100
----------
Training - Loss: 0.0973, Accuracy: 96.8450%
Validation - Loss: 0.3350, Accuracy: 89.6500%
Testing - Loss: 0.3350, Accuracy: 89.6500%
Epoch 63/100
----------
Training - Loss: 0.0941, Accuracy: 96.9500%
Validation - Loss: 0.3245, Accuracy: 89.7800%
Testing - Loss: 0.3245, Accuracy: 89.7800%
Epoch 64/100
----------
Training - Loss: 0.0933, Accuracy: 96.9775%
Validation - Loss: 0.3307, Accuracy: 89.5800%
Testing - Loss: 0.3307, Accuracy: 89.5800%
Epoch 65/100
----------
Training - Loss: 0.0945, Accuracy: 96.8775%
Validation - Loss: 0.3280, Accuracy: 89.8500%
Testing - Loss: 0.3280, Accuracy: 89.8500%
Epoch 66/100
----------
Training - Loss: 0.0923, Accuracy: 97.0000%
Validation - Loss: 0.3137, Accuracy: 90.0300%
Testing - Loss: 0.3137, Accuracy: 90.0300%
Epoch 67/100
----------
Training - Loss: 0.0853, Accuracy: 97.2450%
Validation - Loss: 0.3211, Accuracy: 89.9500%
Testing - Loss: 0.3211, Accuracy: 89.9500%
Epoch 68/100
----------
Training - Loss: 0.0814, Accuracy: 97.3625%
Validation - Loss: 0.3098, Accuracy: 90.4000%
Testing - Loss: 0.3098, Accuracy: 90.4000%
Epoch 69/100
----------
Training - Loss: 0.0857, Accuracy: 97.2000%
Validation - Loss: 0.2954, Accuracy: 90.6800%
Testing - Loss: 0.2954, Accuracy: 90.6800%
Epoch 70/100
----------
Training - Loss: 0.0821, Accuracy: 97.4275%
Validation - Loss: 0.3296, Accuracy: 89.8000%
Testing - Loss: 0.3296, Accuracy: 89.8000%
Epoch 71/100
----------
Training - Loss: 0.0814, Accuracy: 97.4175%
Validation - Loss: 0.3040, Accuracy: 90.5900%
Testing - Loss: 0.3040, Accuracy: 90.5900%
Epoch 72/100
----------
Training - Loss: 0.0806, Accuracy: 97.4625%
Validation - Loss: 0.3026, Accuracy: 90.6200%
Testing - Loss: 0.3026, Accuracy: 90.6200%
Epoch 73/100
----------
Training - Loss: 0.0743, Accuracy: 97.7450%
Validation - Loss: 0.3084, Accuracy: 90.4800%
Testing - Loss: 0.3084, Accuracy: 90.4800%
Epoch 74/100
----------
Training - Loss: 0.0763, Accuracy: 97.6275%
Validation - Loss: 0.3237, Accuracy: 89.9200%
Testing - Loss: 0.3237, Accuracy: 89.9200%
Epoch 75/100
----------
Training - Loss: 0.0745, Accuracy: 97.7300%
Validation - Loss: 0.3419, Accuracy: 89.7200%
Testing - Loss: 0.3419, Accuracy: 89.7200%
Epoch 76/100
----------
Training - Loss: 0.0762, Accuracy: 97.6400%
Validation - Loss: 0.3169, Accuracy: 90.1700%
Testing - Loss: 0.3169, Accuracy: 90.1700%
Epoch 77/100
----------
Training - Loss: 0.0721, Accuracy: 97.7800%
Validation - Loss: 0.3153, Accuracy: 90.2400%
Testing - Loss: 0.3153, Accuracy: 90.2400%
Epoch 78/100
----------
Training - Loss: 0.0691, Accuracy: 97.9650%
Validation - Loss: 0.3160, Accuracy: 90.3400%
Testing - Loss: 0.3160, Accuracy: 90.3400%
Epoch 79/100
----------
Training - Loss: 0.0738, Accuracy: 97.7125%
Validation - Loss: 0.3202, Accuracy: 90.1300%
Testing - Loss: 0.3202, Accuracy: 90.1300%
Epoch 80/100
----------
Training - Loss: 0.0663, Accuracy: 98.0275%
Validation - Loss: 0.3148, Accuracy: 90.4100%
Testing - Loss: 0.3148, Accuracy: 90.4100%
Epoch 81/100
----------
Training - Loss: 0.0398, Accuracy: 98.9900%
Validation - Loss: 0.2468, Accuracy: 91.9100%
Testing - Loss: 0.2468, Accuracy: 91.9100%
Epoch 82/100
----------
Training - Loss: 0.0276, Accuracy: 99.4750%
Validation - Loss: 0.2424, Accuracy: 92.2200%
Testing - Loss: 0.2424, Accuracy: 92.2200%
Epoch 83/100
----------
Training - Loss: 0.0248, Accuracy: 99.5675%
Validation - Loss: 0.2399, Accuracy: 92.3700%
Testing - Loss: 0.2399, Accuracy: 92.3700%
Epoch 84/100
----------
Training - Loss: 0.0235, Accuracy: 99.5900%
Validation - Loss: 0.2451, Accuracy: 92.4900%
Testing - Loss: 0.2451, Accuracy: 92.4900%
Epoch 85/100
----------
Training - Loss: 0.0206, Accuracy: 99.6625%
Validation - Loss: 0.2455, Accuracy: 92.5300%
Testing - Loss: 0.2455, Accuracy: 92.5300%
Epoch 86/100
----------
Training - Loss: 0.0205, Accuracy: 99.6950%
Validation - Loss: 0.2350, Accuracy: 92.6200%
Testing - Loss: 0.2350, Accuracy: 92.6200%
Epoch 87/100
----------
Training - Loss: 0.0193, Accuracy: 99.7600%
Validation - Loss: 0.2394, Accuracy: 92.5800%
Testing - Loss: 0.2394, Accuracy: 92.5800%
Epoch 88/100
----------
Training - Loss: 0.0175, Accuracy: 99.7825%
Validation - Loss: 0.2377, Accuracy: 92.7200%
Testing - Loss: 0.2377, Accuracy: 92.7200%
Epoch 89/100
----------
Training - Loss: 0.0170, Accuracy: 99.7975%
Validation - Loss: 0.2407, Accuracy: 92.5500%
Testing - Loss: 0.2407, Accuracy: 92.5500%
Epoch 90/100
----------
Training - Loss: 0.0174, Accuracy: 99.7725%
Validation - Loss: 0.2393, Accuracy: 92.6200%
Testing - Loss: 0.2393, Accuracy: 92.6200%
Epoch 91/100
----------
Training - Loss: 0.0164, Accuracy: 99.8075%
Validation - Loss: 0.2420, Accuracy: 92.6800%
Testing - Loss: 0.2420, Accuracy: 92.6800%
Epoch 92/100
----------
Training - Loss: 0.0160, Accuracy: 99.8425%
Validation - Loss: 0.2351, Accuracy: 92.5300%
Testing - Loss: 0.2351, Accuracy: 92.5300%
Epoch 93/100
----------
Training - Loss: 0.0159, Accuracy: 99.8100%
Validation - Loss: 0.2374, Accuracy: 92.7300%
Testing - Loss: 0.2374, Accuracy: 92.7300%
Epoch 94/100
----------
Training - Loss: 0.0154, Accuracy: 99.8375%
Validation - Loss: 0.2370, Accuracy: 92.7200%
Testing - Loss: 0.2370, Accuracy: 92.7200%
Epoch 95/100
----------
Training - Loss: 0.0148, Accuracy: 99.8625%
Validation - Loss: 0.2479, Accuracy: 92.6800%
Testing - Loss: 0.2479, Accuracy: 92.6800%
Epoch 96/100
----------
Training - Loss: 0.0141, Accuracy: 99.8900%
Validation - Loss: 0.2395, Accuracy: 92.7600%
Testing - Loss: 0.2395, Accuracy: 92.7600%
Epoch 97/100
----------
Training - Loss: 0.0140, Accuracy: 99.9000%
Validation - Loss: 0.2339, Accuracy: 92.9600%
Testing - Loss: 0.2339, Accuracy: 92.9600%
Epoch 98/100
----------
Training - Loss: 0.0136, Accuracy: 99.8775%
Validation - Loss: 0.2386, Accuracy: 92.6700%
Testing - Loss: 0.2386, Accuracy: 92.6700%
Epoch 99/100
----------
Training - Loss: 0.0139, Accuracy: 99.8775%
Validation - Loss: 0.2450, Accuracy: 92.8100%
Testing - Loss: 0.2450, Accuracy: 92.8100%
Epoch 100/100
----------
Training - Loss: 0.0133, Accuracy: 99.9000%
Validation - Loss: 0.2376, Accuracy: 92.8100%
Testing - Loss: 0.2376, Accuracy: 92.8100%
